<a href="https://colab.research.google.com/github/jsanher/trading-algoritmico-a-z-con-python/blob/main/Moodle_PRO_Grupos_Diplomas_Notas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import csv
import datetime

def AgruparAlumnos(file1):

  # endpoint de la API
  endpoint = "https://formacion.nodofarma.es/moodle/assignGroup"

  # datos en formato JSON a enviar en la llamada PUT
  data = {
      "courseLegacyID":"260015",
      "grouptype":"C"
  }

  # obtener la fecha actual
  now = datetime.datetime.now()
  # Formatear la fecha actual para incluir los segundos
  date_string = now.strftime("%Y-%m-%d %H:%M:%S")
  # crear el nombre del archivo de log con la fecha actual
  log_file = "log_{}.txt".format(date_string)

  # abrir el archivo CSV y leer las filas
  with open(file1, 'r') as file:
      reader = csv.reader(file)
      # abrir el archivo de log
      with open(log_file, "w") as log:
          # iterar sobre las filas del CSV
          for row in reader:
              # obtener el userLegacyID de la fila
              userLegacyID = row[0]
              # agregar el userLegacyID al diccionario de datos JSON
              data["userLegacyID"] = userLegacyID
              # realizar la llamada PUT
              response = requests.put(endpoint, json=data)
              # obtener el status code y descripcion de la respuesta
              status_code = response.status_code
              status_text = response.reason
              # escribir la respuesta en el archivo de log
              log.write("Llamada a {} con userLegacyID {} - Status: {} - {}\n".format(endpoint, userLegacyID, status_code, status_text))
              print("Llamada a {} con userLegacyID {} - Status: {} - {}".format(endpoint, userLegacyID, status_code, status_text))



In [ ]:
AgruparAlumnos("Libro3.csv")

# getScore


In [ ]:
import requests
import csv
import logging
import json

logging.basicConfig(filename='get_scores.log', level=logging.INFO)

def get_scores(course_legacy_id, output_file):
    # get quiz id
    quiz_url = f"https://formacion.nodofarma.es/moodle/listQuizzes?courseLegacyID={course_legacy_id}"
    quiz_response = requests.get(quiz_url)
    quiz_data = quiz_response.json()
    quiz_id = quiz_data['quizzes']
    # get user data
    user_url = f"https://formacion.nodofarma.es/moodle/listCourseUsers?courseLegacyID={course_legacy_id}"
    user_response = requests.get(user_url)
    user_data = user_response.json()
    listausuarios = user_data['users']

    print(listausuarios)

    # write to output file
    with open(output_file, 'w', newline='') as f_output:
        writer = csv.writer(f_output)
        # write header row
        writer.writerow(['user_legacy_id', 'url', 'response'])
        for element in listausuarios:
            user_legacy_id = element
            url = f"https://formacion.nodofarma.es/moodle/getScore?quizID={quiz_id}&userLegacyID={user_legacy_id}"
            response = requests.get(url)
            data = response.json()
            writer.writerow([user_legacy_id, url, data])
            logging.info(f"URL: {url}, Response: {data}")


In [ ]:
get_scores(217, "/content/salida2.csv")


[102581, 76123, 57122, 132067, 137054, 42124, 82082, 128053, 125358, 122379, 20773, 36588, 55104, 149827, 75605, 114693, 43877, 112826, 23124, 100142, 119012, 41617, 51984, 122463, 35973, 48301, 50680, 79803, 86809, 129887, 134521, 135245, 141890, 145002, 7046, 35942, 41070, 41996, 45034, 51433, 56530, 57327, 76020, 100258, 102577, 110273, 127760, 16544, 31618, 39944, 101973, 148515, 1198, 12931, 37754, 46716, 56853, 86124, 122957, 143472, 24116, 29743, 38102, 38161, 39017, 75561, 100187, 111006, 6965, 13193, 10444, 44966, 45239, 56114, 75232, 81775, 100375, 100481, 115214, 120352, 135707, 155422, 23300, 46208, 55116, 123136, 130074, 135693, 151656, 54970, 900281, 85252, 85575, 100738, 107261, 56659, 50351, 101251, 110243, 19665, 39055, 51098, 54056, 81624, 85784, 102725, 107404, 110423, 128149, 147559, 44684, 76247, 106637, 106990, 117372, 133289, 145598, 16356, 36365, 37036, 37431, 81000, 102531, 124963, 140202, 7351, 54313, 82927, 103080, 34176, 109890, 119844, 135276, 136949, 43101

In [ ]:
import csv

def remove_error_rows(file_path):
    # Abrir el archivo CSV original
    with open(file_path, "r") as csv_file:
        # Leer el archivo CSV original
        csv_reader = csv.reader(csv_file)
        
        # Crear una lista para almacenar las líneas que no contienen "error" en la columna "response"
        clean_rows = []
        
        # Recorrer cada línea del archivo CSV original
        for row in csv_reader:
            # Si la columna "response" no contiene "error"
            if "Error" not in row[2]:
                # Añadir la línea a la lista limpia
                clean_rows.append(row)
    
    # Escribir las líneas limpias en un nuevo archivo CSV
    with open("clean_"+file_path, "w", newline='') as clean_file:
        csv_writer = csv.writer(clean_file)
        csv_writer.writerows(clean_rows)
    print(f"Archivo limpio generado con el nombre: clean_{file_path}")


In [ ]:
remove_error_rows("salida2.csv")

Archivo limpio generado con el nombre: clean_salida2.csv


In [ ]:
import csv

def remove_duplicates(file1, file2):
    # Abrir el primer archivo CSV
    with open(file1, "r") as csv_file1:
        csv_reader1 = csv.reader(csv_file1)
        # Guardar la primera columna del primer archivo CSV en una lista
        first_column1 = [row[0] for row in csv_reader1]
    # Abrir el segundo archivo CSV
    with open(file2, "r") as csv_file2:
        csv_reader2 = csv.reader(csv_file2)
        # Crear una lista para almacenar las líneas únicas
        unique_rows = []
        # Recorrer cada línea del segundo archivo CSV
        for row in csv_reader2:
            # Si el campo de la primera columna no existe en el campo de la primera columna del primer archivo CSV
            if row[0] not in first_column1:
                # Añadir la línea a la lista de líneas únicas
                unique_rows.append(row)

    # Escribir las líneas únicas en un nuevo archivo CSV
    with open("unique_rows.csv", "w", newline='') as unique_file:
        csv_writer = csv.writer(unique_file)
        csv_writer.writerows(unique_rows)
    print(f"Archivo con las líneas únicas generado con el nombre: unique_rows.csv")


In [ ]:
remove_duplicates("/content/Libro2.csv","/content/Libro3.csv")

Archivo con las líneas únicas generado con el nombre: unique_rows.csv


In [ ]:
import csv
import requests

url = "https://formacion.nodofarma.es/moodle/setCertificate"
headers = {'Content-Type': 'application/json;charset=UTF-8'}

with open('/content/cuerpo4.csv', newline='') as csvfile:
    data = csv.reader(csvfile)
    next(data) # skip the header row
    for row in data:
        userLegacyID = row[0]
        courseName = row[2]
        urlDiploma = row[3]
        order = row[4]
        courseLegacyID = row[1]

        payload = {
            "userLegacyID": userLegacyID,
            "courseName": courseName,
            "url": urlDiploma,
            "order": order,
            "courseLegacyID": courseLegacyID
        }

        print(payload)
        response = requests.put(url, json=payload, headers=headers)
        print(response.status_code, response.reason) # check if the request was successful
